In [ ]:
#|default_exp export

# PyTest Capabilities with nbprocess

> A notebook allowing for the export of tests in an nbprocess project and thrown into a pytest-compatible file 

In [ ]:
#|export
from nbprocess.process import *
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.maker import *

The goal of this notebook is to demonstrate how easy it can be to add automatic export of tests written inside of a Jupyter Notebook through `nbprocess`.

What does this look like? I'd recommend reading through [00_core](https://github.com/muellerzr/nbprocess-sandbox/blob/main/00_core.ipynb) first to get an idea, but essentially we write one big `unittest.TestCase` for each file (which is commonly done), and then we declare each of our tests through a `#|test {testname}` tag. There is also a special `#|test imports` tag for imports that should be present when performing the testing. 

Importing the exported module via `from library.{module} import *` is done automatically.

In [ ]:
#|export
from collections import defaultdict
from fastcore.foundation import L, ifnone
from execnb.nbio import *

class ExportTestProc:
    "A test proc that watches for `#|default_exp` and `#|test`"
    def __init__(self): self.tests = defaultdict(L)
    def _default_exp_(self, nbp, cell, exp_to): self.default_exp = f'test_{exp_to}'
    def _test_(self, nbp, cell, exp_to=None): self.tests[f'test_{ifnone(exp_to, "#")}'].append(nbp.cell)

In [ ]:
#|export
def write_test_cells(cells, hdr, file, offset=0, name=""):
    "Takes cells and either formats them for import statments, or writes out test functions"
    for cell in cells:
        if cell.source.strip():
            if "import" in cell["directives_"]["test"]:
                # Expected, this should be at the top. What happens is its never written
                content = f'\n{hdr} {cell.idx_+offset}\n{cell.source}'
            else:
                test_name = cell["directives_"]["test"][0]
                content = '\n'.join([f"\t\t{c}" for c in cell.source.split("\n")])
                content = f'\n\t{hdr} {cell.idx_+offset}\n\tdef test_{test_name}(self):\n{content}'
            file.write(content)

In [ ]:
#|export
import ast
class TestMaker(ModuleMaker):
    "Module maker that will write test cells depending on a flag"
    def make_all(self, cells): pass
    def _make_exists(self, cells, all_cells=None):
        with self.fname.open("a") as f: 
            write_test_cells(cells, self.hdr, f, 0, self.name.replace("test_","").capitalize())
    
    def make(self, cells, all_cells=None, lib_name=None, write_start=False):
        if lib_name is None: lib_name = get_config().lib_name
        if all_cells is None: all_cells = cells
        for cell in all_cells: cell.import2relative(lib_name)
        if not self.is_new: return self._make_exists(cells, all_cells)
        
        self.fname.parent.mkdir(exist_ok=True, parents=True)
        trees = cells.map(NbCell.parsed_)
        try: last_future = max(i for i,tree in enumerate(trees) if tree and any(
             isinstance(t,ast.ImportFrom) and t.module=='__future__' for t in tree))+1
        except ValueError: last_future=0
        with self.fname.open('w') as f:
            f.write(f"# AUTOGENERATED! DO NOT EDIT! File to edit: {self.dest2nb}.")
            write_cells(cells[:last_future], self.hdr, f, 0)
            write_test_cells(cells, self.hdr, f, 0, self.name.replace("test_","").capitalize())
            f.write('\n')

In [ ]:
#|export
def create_test_modules(path, dest, debug=False):
    "Creates test module(s) from notebook and saves them to a tests/ folder"
    exp = ExportTestProc()
    nb = NBProcessor(path, exp, debug=debug)
    nb.process()
    for i,(mod, cells) in enumerate(exp.tests.items()):
        name = exp.default_exp
        mm = TestMaker(dest=dest, name=name, nb_path=path, is_new=i==0)
        mm.make(cells, write_start=i==0)
        if i == 0:
            with mm.fname.open("a") as f:
                name = name.lower().replace("test_", "")
                f.write(f'\nfrom {get_config().lib_name}.{name} import *')
                f.write(f'\nimport unittest\n')
                f.write(f'\nclass {name.title()}Tester(unittest.TestCase):')

First we can export the modules and get our `CoreTester` class, checking it has our `test_addition` function:

In [ ]:
#|eval: false
create_test_modules(everything_fn, "tmp")

g = exec_new("from tmp.test_core import CoreTester")
assert hasattr(g["CoreTester"], "test_addition")

Finally we can run the test case and ensure it passes:

In [ ]:
#|eval: false
import unittest
def run_case(testcase:unittest.TestCase):
    "Runs a unittest.TestCase"
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(testcase)
    unittest.TextTestRunner().run(suite)

In [ ]:
#|eval: false
run_case(g["CoreTester"])

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
